# 1. Setup e carregamento de dados

In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
from IPython.display import display

# Configurar o estilo visual para os gráficos
sns.set_style("whitegrid")

# Caminho para o ficheiro de dados processados
caminho_dados = '../data/dados_processados_trimestrais.csv'

try:
    # Carregar o DataFrame, usando a primeira coluna (índice 0) como índice temporal
    df_final = pd.read_csv(caminho_dados, index_col=0, parse_dates=True)
    
    # Atribuir o nome 'data' ao índice para clareza e consistência
    df_final.index.name = 'data'
    
    # Renomear as colunas para nomes mais curtos e fáceis de usar
    df_final.rename(columns={
        'PIB_var_homologa': 'pib',
        'Credito_Empresas_Total': 'credito_empresas',
        'Credito_Particulares_Total': 'credito_particulares',
        'Endividamento_Total': 'endividamento'
    }, inplace=True)

    print("DataFrame carregado e preparado com sucesso!")
    display(df_final.head())

except FileNotFoundError:
    print(f"ERRO: O ficheiro de dados não foi encontrado em '{caminho_dados}'. Verifique o caminho.")

DataFrame carregado e preparado com sucesso!


,pib,credito_empresas,credito_particulares,endividamento
data,,,,
2007-12-31,3.0,4877.6,11.6,156083.31
2008-03-31,1.7,5278.5,11.8,158296.02
2008-06-30,0.9,6077.3,13.0,160952.95
2008-09-30,0.5,6898.4,14.1,162444.83
2008-12-31,-1.9,7738.9,14.7,163756.38


# 2. Centralizar resultados das anomalias

In [ ]:
# Datas de anomalias detetadas pelo IsolationForest (confirmadas)
anomalias_isoforest_datas = [
    '2012-06-30', '2012-09-30', '2012-12-31', # Crise da Dívida Soberana
    '2020-06-30',                             # Choque COVID-19
    '2021-06-30',                             # Recuperação Pós-COVID (efeito base)
    '2022-03-31'                              # Início Crise Inflacionária / Guerra da Ucrânia
]

# Datas de anomalias detetadas pela Decomposição STL (confirmadas, agregadas das 4 séries)
anomalias_stl_datas = [
    '2020-06-30', '2021-06-30',               # Do PIB
    '2011-09-30',                             # Do Crédito a Empresas
    '2017-03-31', '2017-06-30',               # Do Crédito a Particulares
    '2018-12-31', '2019-03-31'                # Do Endividamento
]

# Datas de anomalias detetadas pelo Prophet (confirmadas no PIB)
anomalias_prophet_datas = [
    '2020-06-30', # Queda abrupta (COVID)
    '2021-06-30', # Recuperação muito forte (Efeito Base)
    '2021-12-31', # Pico de recuperação (superior à previsão)
    '2022-09-30'  # Crescimento forte inesperado (superior à previsão)
]

# Converter as listas de strings para objetos DatetimeIndex para correspondência segura
anomalias_isoforest_idx = pd.to_datetime(anomalias_isoforest_datas)
anomalias_stl_idx = pd.to_datetime(anomalias_stl_datas)
anomalias_prophet_idx = pd.to_datetime(anomalias_prophet_datas)

# Inicializar as colunas de anomalias com 0 (não anomalia)
df_final['anomalia_isoforest'] = 0
df_final['anomalia_stl'] = 0
df_final['anomalia_prophet'] = 0

# Marcar com 1 (anomalia) as datas correspondentes
df_final.loc[df_final.index.isin(anomalias_isoforest_idx), 'anomalia_isoforest'] = 1
df_final.loc[df_final.index.isin(anomalias_stl_idx), 'anomalia_stl'] = 1
df_final.loc[df_final.index.isin(anomalias_prophet_idx), 'anomalia_prophet'] = 1

print("Colunas de anomalias adicionadas ao DataFrame.")
print("\nVerificação para confirmar que as colunas foram criadas:")
display(df_final.loc['2020':'2021', ['pib', 'anomalia_isoforest', 'anomalia_stl', 'anomalia_prophet']])

Colunas de anomalias adicionadas ao DataFrame com base nas datas precisas dos modelos.

Verificação das anomalias no período da pandemia (2020-2022):


,pib,anomalia_isoforest,anomalia_stl,anomalia_prophet
data,,,,
2020-03-31,-2.4,0,0,0
2020-06-30,-17.6,1,1,1
2020-09-30,-6.1,0,0,0
2020-12-31,-6.6,0,0,0
2021-03-31,-5.4,0,0,0
2021-06-30,16.5,1,1,1
2021-09-30,5.5,0,0,0
2021-12-31,7.2,0,0,1
2022-03-31,12.2,1,0,0


In [3]:
# 1. Criar a coluna 'contagem_anomalias' somando o número de modelos que detetaram uma anomalia em cada data
df_final['contagem_anomalias'] = df_final[['anomalia_isoforest', 'anomalia_stl', 'anomalia_prophet']].sum(axis=1)

# 2. Criar um novo DataFrame 'df_comparativo' contendo apenas as linhas onde pelo menos uma anomalia foi detetada
df_comparativo = df_final[df_final['contagem_anomalias'] > 0].copy()

# 3. Ordenar a tabela: primeiro pelos eventos com maior consenso (maior contagem de anomalias), depois pela data
df_comparativo.sort_values(by=['contagem_anomalias', 'data'], ascending=[False, True], inplace=True)

# 4. Selecionar e reordenar as colunas para uma apresentação clara e fácil de interpretar
colunas_para_exibir = [
    'pib', 'credito_empresas', 'credito_particulares', 'endividamento',
    'anomalia_isoforest', 'anomalia_stl', 'anomalia_prophet', 'contagem_anomalias'
]
df_comparativo = df_comparativo[colunas_para_exibir]

print("Tabela Comparativa de Anomalias gerada com sucesso!")
print("\nEsta tabela resume todos os eventos anómalos detetados pelos 3 modelos:")
display(df_comparativo)

Tabela Comparativa de Anomalias gerada com sucesso!

Esta tabela resume todos os eventos anómalos detetados pelos 3 modelos:


,pib,credito_empresas,credito_particulares,endividamento,anomalia_isoforest,anomalia_stl,anomalia_prophet,contagem_anomalias
data,,,,,,,,
2020-06-30,-17.6,8896.8,18.8,143217.50,1,1,1,3
2021-06-30,16.5,7105.6,16.5,147519.12,1,1,1,3
2011-09-30,-2.3,19208.8,27.0,165525.90,0,1,0,1
2012-06-30,-4.0,27934.1,32.1,160264.04,1,0,0,1
2012-09-30,-4.3,31060.9,33.5,158446.81,1,0,0,1
2012-12-31,-4.6,31471.3,34.7,156432.13,1,0,0,1
2017-03-31,3.7,34215.1,31.0,135192.51,0,1,0,1
2017-06-30,3.5,32723.8,37.6,134992.61,0,1,0,1
2018-12-31,2.9,19948.5,26.9,136294.43,0,1,0,1
